## Introduction

The code is part of a utility module designed to support experiments in a deep learning project. It includes functions and classes that help manage experiment directories, set random seeds for reproducibility, display progress bars, and compute performance metrics. These utilities are essential for organizing experiments, ensuring consistent results, and monitoring progress and performance.

## Functions and Classes

### 1. Function: `make_exp_dirs`

**Purpose:** Creates necessary directories for experiments based on the provided configuration options. It checks whether the mode is training or testing and creates directories accordingly, with options to overwrite existing directories if in debug mode.

**Key Operations:**

* Copies path options to avoid modifying the original dictionary.
* Creates directories for experiments and models if in training mode.
* Creates a results directory if not in training mode.

```python
def make_exp_dirs(opt):
    """Make directories for experiments."""
    path_opt = opt['path'].copy()
    if opt['is_train']:
        overwrite = True if 'debug' in opt['name'] else False
        os.makedirs(path_opt.pop('experiments_root'), exist_ok=overwrite)
        os.makedirs(path_opt.pop('models'), exist_ok=overwrite)
    else:
        os.makedirs(path_opt.pop('results_root'))  

### 2. Function: `set_random_seed`

**Purpose:** Sets random seeds for various libraries to ensure reproducibility of experiments. This is crucial in machine learning to achieve consistent results across different runs.

**Key Operations:**

* Sets seeds for Python's `random` module, NumPy, and PyTorch (including CUDA).

```python
def set_random_seed(seed):
    """Set random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

### 3. Class: `ProgressBar`

**Purpose:** Displays a progress bar in the terminal to visually track the completion of iterative tasks, such as training epochs or data loading. This provides immediate feedback to the user about the ongoing process and its estimated remaining time.

**Key Methods:**

* `__init__(num_batches, bar_width=50)`: Initializes the `ProgressBar` instance.
    * `num_batches`: The total number of iterations or steps in the process to be tracked.
    * `bar_width`: The desired width (in characters) of the progress bar display. Defaults to 50.
* `_get_max_bar_width()`: Determines the maximum usable width for the progress bar by considering the current terminal size. This ensures the bar doesn't overflow the terminal.
* `start()`: Initializes and displays the empty progress bar at the beginning of the process.
* `update(current, epoch=None, total_epoch=None, loss=None, lr=None)`: Updates the progress bar as each step is completed.
    * `current`: The current iteration number (0-indexed).
    * `epoch` (optional): The current epoch number, if applicable.
    * `total_epoch` (optional): The total number of epochs, if applicable.
    * `loss` (optional): The current loss value to display.
    * `lr` (optional): The current learning rate to display.
    It calculates and displays the percentage complete, elapsed time, and estimated time remaining (ETA). Optionally, it can also display the current epoch, loss, and learning rate.

### 4. Class: `AverageMeter`

**Purpose:** Computes and stores the average and current value of a numerical metric over a series of updates. This is commonly used in machine learning to track performance indicators like loss, accuracy, or other relevant metrics during training or evaluation.

**Key Methods:**

* `__init__()`: Initializes an `AverageMeter` object, setting the initial values of `val`, `avg`, `sum`, and `count` to zero.
* `reset()`: Resets all the stored values (`val`, `avg`, `sum`, `count`) to their initial state (zero). This is useful for starting the calculation of an average for a new epoch or evaluation phase.
* `update(val, n=1)`: Updates the meter with a new value.
    * `val`: The new value of the metric.
    * `n` (optional): The number of samples that this value represents (defaults to 1). This is useful when the value is an average over multiple samples.
    The method updates the running `sum` of the values, the total `count` of samples, the `current` value (`val`), and recalculates the `average` (`avg`) based on the updated sum and count.

## import the necessary libraries

In [5]:
import logging # Import the logging module for outputting messages during program execution.
import os # Import the os module for interacting with the operating system, such as checking file paths.
import random # Import the random module for generating random numbers, often used for initialization or data shuffling.    
import sys # Import the sys module for accessing system-specific parameters and functions, like exiting the program.
import time # Import the time module for time-related operations, such as measuring execution time.
from shutil import get_terminal_size # Import the get_terminal_size function from the shutil module to get the dimensions of the terminal.

# Import the NumPy library for numerical operations, especially for handling arrays and matrices.
import numpy as np
# Import the PyTorch library
import torch
# Get a logger instance named 'base'. This logger can be used throughout the project
# to output messages. It's common practice to have a base logger.
logger = logging.getLogger('base')

## 1. Function: make_exp_dirs
### Purpose: Create necessary directories for experiments based on configuration options.


In [ ]:
def make_exp_dirs(opt):
    """Create directories for experiments based on the provided options.

    Args:
        opt (dict): A dictionary containing configuration options. It should
                    include a 'path' key with sub-keys for directory paths,
                    and an 'is_train' key to indicate if the mode is training.
    """
    # Copy the path options to avoid modifying the original dictionary
    path_opt = opt['path'].copy()

    # Check if the mode is training
    if opt['is_train']:
        # Determine if directories should be overwritten based on the experiment name
        overwrite = True if 'debug' in opt['name'] else False

        # Create the 'experiments_root' directory, allowing overwrite if in debug mode
        os.makedirs(path_opt.pop('experiments_root'), exist_ok=overwrite)

        # Create the 'models' directory, allowing overwrite if in debug mode
        os.makedirs(path_opt.pop('models'), exist_ok=overwrite)
    else:
        # If not in training mode, create the 'results_root' directory
        os.makedirs(path_opt.pop('results_root'))

#### 1. example usage

opt = {
    'path': {
        'experiments_root': './experiments',
        'models': './models',
        'results_root': './results'
    },
    'is_train': True,
    'name': 'experiment_debug'
}

make_exp_dirs(opt)

In [ ]:
# def make_exp_dirs(opt):
#     """Make dirs for experiments."""
#     path_opt = opt['path'].copy()
#     if opt['is_train']:
#         overwrite = True if 'debug' in opt['name'] else False
#         os.makedirs(path_opt.pop('experiments_root'), exist_ok=overwrite)
#         os.makedirs(path_opt.pop('models'), exist_ok=overwrite)
#     else:
#         os.makedirs(path_opt.pop('results_root'))

## set seed for reproducibility

In [6]:
def set_random_seed(seed):
    """Set random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

2. exmaple usage

set_random_seed(42)

## Class: ProgressBar
### Purpose: A class to display a progress bar in the terminal, useful for tracking the progress of tasks.


In [ ]:
# Class: ProgressBar
# Purpose: A class to display a progress bar in the terminal, useful for tracking the progress of tasks.

class ProgressBar(object):
    """A progress bar which can print the progress.

    Modified from:
    https://github.com/hellock/cvbase/blob/master/cvbase/progress.py
    """

    def __init__(self, task_num=0, bar_width=50, start=True):
        # Initialize the number of tasks and the width of the progress bar
        self.task_num = task_num
        # Determine the maximum width of the progress bar based on terminal size
        max_bar_width = self._get_max_bar_width()
        # Set the bar width, ensuring it does not exceed the maximum
        self.bar_width = (
            bar_width if bar_width <= max_bar_width else max_bar_width)
        # Initialize the count of completed tasks
        self.completed = 0
        # Start the progress bar if specified
        if start:
            self.start()

    def _get_max_bar_width(self):
        # Get the current terminal size
        terminal_width, _ = get_terminal_size()
        # Calculate the maximum bar width based on terminal size
        max_bar_width = min(int(terminal_width * 0.6), terminal_width - 50)
        # Ensure the bar width is not too small
        if max_bar_width < 10:
            print(f'terminal width is too small ({terminal_width}), '
                  'please consider widen the terminal for better '
                  'progressbar visualization')
            max_bar_width = 10
        return max_bar_width

    def start(self):
        # Display the initial state of the progress bar
        if self.task_num > 0:
            sys.stdout.write(f"[{' ' * self.bar_width}] 0/{self.task_num}, "
                             f'elapsed: 0s, ETA:\nStart...\n')
        else:
            sys.stdout.write('completed: 0, elapsed: 0s')
        sys.stdout.flush()
        # Record the start time
        self.start_time = time.time()

    def update(self, msg='In progress...'):
        # Increment the count of completed tasks
        self.completed += 1
        # Calculate the elapsed time
        elapsed = time.time() - self.start_time
        # Calculate the tasks per second
        fps = self.completed / elapsed
        if self.task_num > 0:
            # Calculate the percentage of tasks completed
            percentage = self.completed / float(self.task_num)
            # Estimate the time remaining
            eta = int(elapsed * (1 - percentage) / percentage + 0.5)
            # Calculate the number of completed marks in the bar
            mark_width = int(self.bar_width * percentage)
            # Create the bar string
            bar_chars = '>' * mark_width + '-' * (self.bar_width - mark_width)
            # Move the cursor up 2 lines
            sys.stdout.write('\033[2F')
            # Clear the line
            sys.stdout.write('\033[J')
            # Display the updated progress bar
            sys.stdout.write(
                f'[{bar_chars}] {self.completed}/{self.task_num}, '
                f'{fps:.1f} task/s, elapsed: {int(elapsed + 0.5)}s, '
                f'ETA: {eta:5}s\n{msg}\n')
        else:
            # Display the completed tasks and elapsed time
            sys.stdout.write(
                f'completed: {self.completed}, elapsed: {int(elapsed + 0.5)}s, '
                f'{fps:.1f} tasks/s')
        sys.stdout.flush()

3. Example usage

progress = ProgressBar(task_num=100)

for i in range(100):
    # Simulate some work
    time.sleep(0.1)
    progress.update()


###### Initializes a progress bar with 100 tasks.
###### Updates the progress bar in each iteration, showing the percentage completed, elapsed time, and estimated time remaining.

In [3]:
# class ProgressBar(object):
#     """A progress bar which can print the progress.

#     Modified from:
#     https://github.com/hellock/cvbase/blob/master/cvbase/progress.py
#     """

#     def __init__(self, task_num=0, bar_width=50, start=True):
#         self.task_num = task_num
#         max_bar_width = self._get_max_bar_width()
#         self.bar_width = (
#             bar_width if bar_width <= max_bar_width else max_bar_width)
#         self.completed = 0
#         if start:
#             self.start()

#     def _get_max_bar_width(self):
#         terminal_width, _ = get_terminal_size()
#         max_bar_width = min(int(terminal_width * 0.6), terminal_width - 50)
#         if max_bar_width < 10:
#             print(f'terminal width is too small ({terminal_width}), '
#                   'please consider widen the terminal for better '
#                   'progressbar visualization')
#             max_bar_width = 10
#         return max_bar_width

#     def start(self):
#         if self.task_num > 0:
#             sys.stdout.write(f"[{' ' * self.bar_width}] 0/{self.task_num}, "
#                              f'elapsed: 0s, ETA:\nStart...\n')
#         else:
#             sys.stdout.write('completed: 0, elapsed: 0s')
#         sys.stdout.flush()
#         self.start_time = time.time()

#     def update(self, msg='In progress...'):
#         self.completed += 1
#         elapsed = time.time() - self.start_time
#         fps = self.completed / elapsed
#         if self.task_num > 0:
#             percentage = self.completed / float(self.task_num)
#             eta = int(elapsed * (1 - percentage) / percentage + 0.5)
#             mark_width = int(self.bar_width * percentage)
#             bar_chars = '>' * mark_width + '-' * (self.bar_width - mark_width)
#             sys.stdout.write('\033[2F')  # cursor up 2 lines
#             sys.stdout.write(
#                 '\033[J'
#             )  # clean the output (remove extra chars since last display)
#             sys.stdout.write(
#                 f'[{bar_chars}] {self.completed}/{self.task_num}, '
#                 f'{fps:.1f} task/s, elapsed: {int(elapsed + 0.5)}s, '
#                 f'ETA: {eta:5}s\n{msg}\n')
#         else:
#             sys.stdout.write(
#                 f'completed: {self.completed}, elapsed: {int(elapsed + 0.5)}s, '
#                 f'{fps:.1f} tasks/s')
#         sys.stdout.flush()

## 4. Class: AverageMeter
### Purpose: Computes and stores the average and current value of a metric, useful for tracking performance metrics over time.


In [ ]:
# Class: AverageMeter
# Purpose: Computes and stores the average and current value of a metric, useful for tracking performance metrics over time.

class AverageMeter(object):
    """
    Computes and stores the average and current value.
    Imported from:
    https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """

    def __init__(self):
        # Initialize the AverageMeter by resetting its values
        self.reset()

    def reset(self):
        # Reset all metrics to their initial state
        self.val = 0    # Current value
        self.avg = 0    # Running average
        self.sum = 0    # Running sum of all values
        self.count = 0  # Count of values added

    def update(self, val, n=1):
        """
        Update the meter with a new value.

        Args:
            val (float): The new value to add.
            n (int): The weight of the new value, typically the batch size.
        """
        # Update the sum with the new value weighted by n
        self.sum += val * n
        # Update the count with the weight n
        self.count += n
        # Calculate the new average
        self.avg = self.sum / self.count
        

## 4. example usage: To track the average loss over several batches:
loss_meter = AverageMeter()

for batch_loss in [0.5, 0.4, 0.6, 0.3]:
    loss_meter.update(batch_loss)

print(f"Average Loss: {loss_meter.avg}")


### 4. example usage: Computes and stores the average and current value of a metric (e.g., loss).


In [4]:
# class AverageMeter(object):
#     """
#     Computes and stores the average and current value
#     Imported from
#     https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
#     """

#     def __init__(self):
#         self.reset()

#     def reset(self):
#         self.val = 0
#         self.avg = 0  # running average = running sum / running count
#         self.sum = 0  # running sum
#         self.count = 0  # running count

#     def update(self, val, n=1):
#         # n = batch_size

#         # val = batch accuracy for an attribute
#         # self.val = val

#         # sum = 100 * accumulative correct predictions for this attribute
#         self.sum += val * n

#         # count = total samples so far
#         self.count += n

#         # avg = 100 * avg accuracy for this attribute
#         # for all the batches so far
#         self.avg = self.sum / self.count
